In [6]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = "https://courses.analyticsvidhya.com/collections/courses?page="

with open('free_courses_vidhya.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Course Name', 'Lessons', 'Price', 'Link'])

    page = 1
    while True:
        response = requests.get(base_url + str(page))
        soup = BeautifulSoup(response.content, 'html.parser')

        courses = soup.find_all('a', class_='course-card')

        if not courses:
            break

        for course in courses:
            course_name = course.find('h3').text.strip() if course.find('h3') else 'N/A'
            lessons = course.find('span', class_='course-card__lesson-count')
            lessons = lessons.text.strip() if lessons else 'N/A'
            price = course.find('span', class_='course-card__price')
            price = price.text.strip() if price else 'N/A'
            link = "https://courses.analyticsvidhya.com" + course['href'] if course.get('href') else 'N/A'

            writer.writerow([course_name, lessons, price, link])

        page += 1

print("Scraping complete. Data saved to 'free_courses_vidhya.csv'.")


Scraping complete. Data saved to 'free_courses_vidhya.csv'.


In [8]:
import json

with open('free_courses_vidhya.csv', 'r') as file:
    data = [line.strip().split(',') for line in file.readlines()][1:]

courses_data = [{"course_name": course[0], "lessons": course[1], "price": course[2], "link": course[3]} for course in data]

with open('free_courses_vidhya.json', 'w') as json_file:
    json.dump(courses_data, json_file)

print("Data saved as JSON.")


Data saved as JSON.


In [12]:
import json

with open('free_courses_vidhya.json', 'r') as file:
    courses_data = json.load(file)

for course in courses_data[:10]:
    print(course)


{'course_name': 'Frameworks for Effective Problem Solving', 'lessons': '18 Lessons', 'price': 'Free', 'link': 'https://courses.analyticsvidhya.com/courses/frameworks-for-effective-problem-solving'}
{'course_name': 'Anyone can Build AI Agents - Free Course', 'lessons': '5 Lessons', 'price': 'Free', 'link': 'https://courses.analyticsvidhya.com/courses/your-ultimate-guide-to-becoming-an-agentic-ai-expert-by-2025'}
{'course_name': 'A Comprehensive Learning Path to Become a Data Analyst in 2025', 'lessons': '298 Lessons', 'price': 'Free', 'link': 'https://courses.analyticsvidhya.com/courses/a-comprehensive-learning-path-to-become-a-data-analyst-in-2025'}
{'course_name': 'Reimagining GenAI: Common Mistakes and Best Practices for Success', 'lessons': '6 Lessons', 'price': 'Free', 'link': 'https://courses.analyticsvidhya.com/courses/reimagining-genai-common-mistakes-and-best-practices-for-success'}
{'course_name': 'Coding a ChatGPT-style Language Model from Scratch in PyTorch', 'lessons': '7 L

In [20]:
course_names = [course['course_name'] for course in courses_data]
lessons = [course['lessons'] for course in courses_data]
prices = [course['price'] for course in courses_data]
links = [course['link'] for course in courses_data]

print(course_names[:5])
print(lessons[:5])
print(prices[:5])
print(links[:5])


['Frameworks for Effective Problem Solving', 'Anyone can Build AI Agents - Free Course', 'A Comprehensive Learning Path to Become a Data Analyst in 2025', 'Reimagining GenAI: Common Mistakes and Best Practices for Success', 'Coding a ChatGPT-style Language Model from Scratch in PyTorch']
['18 Lessons', '5 Lessons', '298 Lessons', '6 Lessons', '7 Lessons']
['Free', 'Free', 'Free', 'Free', 'Free']
['https://courses.analyticsvidhya.com/courses/frameworks-for-effective-problem-solving', 'https://courses.analyticsvidhya.com/courses/your-ultimate-guide-to-becoming-an-agentic-ai-expert-by-2025', 'https://courses.analyticsvidhya.com/courses/a-comprehensive-learning-path-to-become-a-data-analyst-in-2025', 'https://courses.analyticsvidhya.com/courses/reimagining-genai-common-mistakes-and-best-practices-for-success', 'https://courses.analyticsvidhya.com/courses/coding-a-chatgpt-style-language-model-from-scratch-in-pytorch']


In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

course_embeddings = model.encode(course_names)

print(course_embeddings[:5])


[[ 0.0207321  -0.00083077 -0.0013544  ...  0.03848465  0.05758337
   0.0582363 ]
 [-0.04901564 -0.09395265 -0.04070522 ... -0.00400748  0.02121344
  -0.03748144]
 [-0.00795452 -0.04323222 -0.00785497 ... -0.04750539 -0.06105438
   0.01614271]
 [-0.04188452  0.07991543 -0.01995979 ...  0.031822   -0.05563084
   0.00499994]
 [-0.07910202 -0.08839826  0.01572709 ...  0.103826    0.01209753
  -0.02748035]]


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

def search_courses(query, course_names, course_embeddings, top_n=5):
    query_embedding = model.encode([query])
    similarities = cosine_similarity(query_embedding, course_embeddings)
    top_indices = similarities[0].argsort()[-top_n:][::-1]
    results = [(course_names[i], similarities[0][i]) for i in top_indices]
    
    return results

# Example search query
query = "Data analysis course"
results = search_courses(query, course_names, course_embeddings)

# Print the results
for course, score in results:
    print(f"Course: {course}, Similarity Score: {score:.4f}")


Course: A Comprehensive Learning Path to Become a Data Analyst in 2025, Similarity Score: 0.6695
Course: Data Science Career Conclave, Similarity Score: 0.6485
Course: Introductory Data Science for Business Managers, Similarity Score: 0.6233
Course: A Comprehensive Learning Path to Become a Data Scientist in 2024, Similarity Score: 0.6063
Course: Linear Programming for Data Science Professionals, Similarity Score: 0.5631


In [26]:
import faiss
import numpy as np

embedding_matrix = np.array(course_embeddings).astype('float32')

index = faiss.IndexFlatL2(embedding_matrix.shape[1])
index.add(embedding_matrix)

faiss.write_index(index, 'free_course_index.faiss')


In [28]:
def search_courses(query, top_k=5):
    query_embedding = model.encode([query])
    
    similarities = cosine_similarity(query_embedding, course_embeddings)
    
    indices = similarities.argsort()[0][-top_k:][::-1]
    
    results = []
    for idx in indices:
        course = courses_data[idx]
        results.append({
            'Course Name': course['course_name'],  
            'Lessons': course['lessons'],
            'Price': course['price'],
            'Link': course['link']  
        })
    
    return results

# Example: Search for a course related to "Python"
query = "Python"
search_results = search_courses(query)

for result in search_results:
    print(result)


{'Course Name': 'Introduction to Python', 'Lessons': '69 Lessons', 'Price': 'Free', 'Link': 'https://courses.analyticsvidhya.com/courses/introduction-to-data-science'}
{'Course Name': 'Machine Learning Starter Program', 'Lessons': 'N/A', 'Price': 'Free', 'Link': 'https://courses.analyticsvidhya.com/bundles/machine-learning-starter-program'}
{'Course Name': 'Introduction to Web Scraping using Python', 'Lessons': '13 Lessons', 'Price': 'Free', 'Link': 'https://courses.analyticsvidhya.com/courses/introduction-to-web-scraping'}
{'Course Name': 'Pandas for Data Analysis in Python', 'Lessons': '27 Lessons', 'Price': 'Free', 'Link': 'https://courses.analyticsvidhya.com/courses/pandas-for-data-analysis-in-python'}
{'Course Name': 'Loan Prediction Practice Problem (Using Python)', 'Lessons': '16 Lessons', 'Price': 'Free', 'Link': 'https://courses.analyticsvidhya.com/courses/loan-prediction-practice-problem-using-python'}
